# Neo4j Crud Operations With Python

## NoSQL Final Project
Walid El Otmani - Graph Database for a Brazilian E-Commerce Store

<p>In this notebook we will apply the CRUD operations for a specific customer and save the recommendation system as a function. Python was used to simulate how a real backend program interacts with Neo4j.</p>
<p>Instead of manually running queries in Neo4j Browser, the project uses the Neo4j Python Driver inside Jupyter Notebooks to perform create, read, update, and delete actions programmatically.</p>
<p>First we start off by importing the packages from neo4j, and creating a connection to the database:</p>

In [2]:
from neo4j import GraphDatabase


URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "password"  

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

# Creating a function to run a Cypher query
def run_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params or {})
        return [r.data() for r in result]

print("Connected to Neo4j!")

Connected to Neo4j!


## Create
Now that we connected to the database, let's start with the CRUD Operations. 

In [3]:
# Create a function to create a customer
def create_customer(customer_id, zip_prefix, city, state):
    query = """
    CREATE (c:Customer {
        customer_id: $customer_id, 
        customer_zip_code_prefix: $zip,
        customer_city: $city,
        customer_state: $state
    })
    RETURN c
    """
    return run_query(query, {
        "customer_id": customer_id,
        "zip": zip_prefix,
        "city": city,
        "state": state
    })

# Test
create_customer("TEST123", 12345, "TestCity", "TS")

[{'c': {'customer_state': 'TS',
   'customer_id': 'TEST123',
   'customer_zip_code_prefix': 12345,
   'customer_city': 'TestCity'}}]

## Read
Now that we created a customer, we can test a Reading operation:

In [4]:
# Create a function to read a customer by customer_id
def read_customer(customer_id):
    query = """
    MATCH(c:Customer {customer_id: $customer_id})
    RETURN c
    """
    return run_query(query,{"customer_id": customer_id})

#Testing the read function
read_customer("TEST123")

[{'c': {'customer_state': 'TS',
   'customer_id': 'TEST123',
   'customer_zip_code_prefix': 12345,
   'customer_city': 'TestCity'}}]

## Update
Updating a customer's city

In [5]:
# Create a function to update a customer's city 
def update_customer_city(customer_id, new_city):
    query = """
    MATCH(c:Customer {customer_id: $customer_id})
    SET c.customer_city = $new_city
    RETURN c
    """
    return run_query(query, {
        "customer_id": customer_id,
        "new_city": new_city
    })

# Testing the update function
update_customer_city("TEST123", "NewTestCity")

[{'c': {'customer_state': 'TS',
   'customer_id': 'TEST123',
   'customer_city': 'NewTestCity',
   'customer_zip_code_prefix': 12345}}]

## Delete
Deleting a customer


In [6]:
# Create a function to delete a customer using customer_id
def delete_customer(customer_id):
    query = """
    MATCH(c:Customer {customer_id: $customer_id})
    DELETE c
    """
    return run_query(query, {"customer_id": customer_id})

# Testing the delete Function
delete_customer("TEST123")

# Verify Deletion
read_customer("TEST123")  # Returns an empty list if deletion was successful

[]

## Recommendation System
Python can also be used to save the recommendation engine in a function, which will be later used as an important part of the backend of the E-commerce store.

In [7]:
def recommend_products(product_id, limit=3):
    query = """
    MATCH (p:Product {product_id: $productId})
    WHERE p.in_demo = true
    
    MATCH (p)<-[:CONTAINS_PRODUCT]-(oi:OrderItem)
          <-[:HAS_ITEM]-(o:Order)
    MATCH (o)-[:HAS_ITEM]->(otherOi:OrderItem)
          -[:CONTAINS_PRODUCT]->(other:Product)
    WHERE other.product_id <> p.product_id
      AND other.in_demo = true

    WITH other, COUNT(DISTINCT o) AS coPurchaseCount
    RETURN 
      other.product_id AS id,
      coalesce(other.display_name, other.product_category_name + ' ' + substring(other.product_id, 0, 6)) AS name,
      other.product_category_name AS category,
      coPurchaseCount
    ORDER BY coPurchaseCount DESC
    LIMIT $limit
    """
    return run_query(query, {"productId": product_id, "limit": limit})